In [111]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from functools import reduce
from datetime import datetime
from pyNBA.Models.features import FeatureCreation
from pyNBA.Data.data import QueryData
from pyNBA.Models.helpers import CleanData
from research import Helpers
from pyNBA.Models.StatsV2Backtest.minutesplayed import MinutesPlayed
from pyNBA.Models.StatsV2Backtest.possessionsperminute import PossessionsPerMinute
from pyNBA.Models.StatsV2Backtest.pointsperpossession import PointsPerPossession
from pyNBA.Models.StatsV2Backtest.assistsperpossession import AssistsPerPossession
from pyNBA.Models.StatsV2Backtest.reboundsperpossession import ReboundsPerPossession
from pyNBA.Models.StatsV2Backtest.turnoversperpossession import TurnoversPerPossession

import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
query_data = QueryData(update=False)
clean_data = CleanData()

boxscores = query_data.query_boxscore_data()
boxscores = boxscores.loc[boxscores['SEASONTYPE'] == 'Regular Season']
boxscores = clean_data.drop_rows_player_injured(boxscores)

/Users/brandonshimiaie/Projects/pyNBA/sqlite/db/nba.db
2.6.0


In [3]:
start_date = boxscores['DATE'].min()
end_date = boxscores['DATE'].max()

print('generating minutes data...')
mp_out = MinutesPlayed().predict(boxscores, start_date, end_date, update=False)

print('generating possessions/minute data...')
ppm_out = PossessionsPerMinute().predict(boxscores, start_date, end_date)

print('generating points/possession data...')
ppp_out = PointsPerPossession().predict(boxscores, start_date, end_date)

print('generating rebounds/possession data...')
rpp_out = ReboundsPerPossession().predict(boxscores, start_date, end_date)

print('generating assists/possession data...')
app_out = AssistsPerPossession().predict(boxscores, start_date, end_date)

print('generating turnovers/possession data...')
tpp_out = TurnoversPerPossession().predict(boxscores, start_date, end_date)

generating minutes data...
/Users/brandonshimiaie/Projects/pyNBA/sqlite/db/nba.db
2.6.0
generating possessions/minute data...
generating points/possession data...
generating rebounds/possession data...
generating assists/possession data...
generating turnovers/possession data...


In [4]:
predicted_boxscores = reduce(
    lambda left, right: pd.merge(
        left, right, on=['SEASON', 'DATE', 'TEAM', 'OPP_TEAM', 'PLAYERID', 'START']
        ), [mp_out, ppm_out, ppp_out, app_out, rpp_out, tpp_out]
        )

In [120]:
team_boxscores = predicted_boxscores.groupby(['SEASON', 'DATE', 'TEAM', 'OPP_TEAM']).apply(
    lambda x: pd.Series({
        'MED_TEAM_POSS/MP': x['POSS/MP'].median(),
        'MIN_STARTER_MP': x.loc[x['START'] == 1]['AVG_MP(REG)_R'].min(),
        'MED_BENCH_MP': x.loc[x['START'] == 0]['AVG_MP(REG)_R'].median(),
        'MED_TEAM_OREB/CH': x['OREB/CH'].median(),
        'MED_TEAM_DREB/CH': x['DREB/CH'].median(),
        'MIN_STARTER_A/P': x.loc[x['START'] == 1]['ATT/POSS'].min(),
        'MED_BENCH_A/P': x.loc[x['START'] == 0]['ATT/POSS'].median(),
        'MED_TEAM_TOV/POSS': x['TOV/POSS'].median(),
        'TEAM_OREB(CH)/POSS': x['OREB(CH)/POSS'].mean(),
        'TEAM_OREB(CH)/POSS_DEF': x['OREB(CH)/POSS_DEF'].mean()
    })
).reset_index()

opp_team_boxscores = team_boxscores.drop(columns=[
    'OPP_TEAM', 'MED_TEAM_POSS/MP', 'MIN_STARTER_MP', 'MED_BENCH_MP', 'MED_TEAM_OREB/CH', 'MED_TEAM_DREB/CH',
    'MIN_STARTER_A/P', 'MED_BENCH_A/P', 'MED_TEAM_TOV/POSS'
])
opp_team_boxscores = opp_team_boxscores.rename(columns={
    'TEAM': 'OPP_TEAM', 'TEAM_OREB(CH)/POSS': 'OPP_TEAM_OREB(CH)/POSS', 'TEAM_OREB(CH)/POSS_DEF': 'OPP_TEAM_OREB(CH)/POSS_DEF'
    })
team_boxscores = team_boxscores.merge(opp_team_boxscores, on=['SEASON', 'DATE', 'OPP_TEAM'], how='left')

In [121]:
season_stats = predicted_boxscores.groupby(['SEASON']).apply(
    lambda x: pd.Series({
        'SEASON_PTS/ATT': x['PTS/ATT'].median()
    })
).reset_index()

In [122]:
temp = predicted_boxscores.merge(team_boxscores, on=['SEASON', 'DATE', 'TEAM', 'OPP_TEAM'], how='left')
temp = temp.merge(season_stats, on=['SEASON'], how='left')

In [123]:
temp['POSS/MP'] = temp['POSS/MP'].fillna(temp['MED_TEAM_POSS/MP'])
temp.loc[temp['START'] == 1, 'AVG_MP(REG)_R'] = temp.loc[temp['START'] == 1, 'AVG_MP(REG)_R'].fillna(temp['MIN_STARTER_MP'])
temp.loc[temp['START'] == 0, 'AVG_MP(REG)_R'] = temp.loc[temp['START'] == 0, 'AVG_MP(REG)_R'].fillna(temp['MED_BENCH_MP'])
temp['OREB/CH'] = temp['OREB/CH'].fillna(temp['MED_TEAM_OREB/CH'])
temp['DREB/CH'] = temp['DREB/CH'].fillna(temp['MED_TEAM_DREB/CH'])
temp.loc[temp['START'] == 1, 'ATT/POSS'] = temp.loc[temp['START'] == 1, 'ATT/POSS'].fillna(temp['MIN_STARTER_A/P'])
temp.loc[temp['START'] == 0, 'ATT/POSS'] = temp.loc[temp['START'] == 0, 'ATT/POSS'].fillna(temp['MED_BENCH_A/P'])
temp['TOV/POSS'] = temp['TOV/POSS'].fillna(temp['MED_TEAM_TOV/POSS'])
temp['PTS/ATT'] = temp['PTS/ATT'].fillna(temp['SEASON_PTS/ATT'])

In [124]:
temp['POSS/MP_REAL'] = (temp['POSS/MP'] + temp['POSS/MP_DEF'])/2
temp['OREB(CH)/POSS_REAL'] = (temp['TEAM_OREB(CH)/POSS'] + temp['TEAM_OREB(CH)/POSS_DEF'])/2
temp['OPP_OREB(CH)/POSS_REAL'] = (temp['OPP_TEAM_OREB(CH)/POSS'] + temp['OPP_TEAM_OREB(CH)/POSS_DEF'])/2
temp['OREB/CH_REAL'] = temp['OREB/CH'] * temp['OREB/CH_DEF']
temp['DREB/CH_REAL'] = temp['DREB/CH'] * temp['DREB/CH_DEF']

In [125]:
aggregated_stats = temp.groupby(['SEASON', 'DATE', 'TEAM', 'OPP_TEAM']).apply(
    lambda x: pd.Series({
        'TEAM_MP': x['AVG_MP(REG)_R'].sum()
    })
).reset_index()

In [126]:
temp = temp.merge(aggregated_stats, on=['SEASON', 'DATE', 'TEAM', 'OPP_TEAM'], how='left')

In [127]:
temp['MP_REAL'] = 240*temp['AVG_MP(REG)_R']/temp['TEAM_MP']
temp['IMPLIED_POSS'] = temp['MP_REAL'] * temp['POSS/MP_REAL']

In [128]:
aggregated_stats = temp.groupby(['SEASON', 'DATE', 'TEAM', 'OPP_TEAM']).apply(
    lambda x: pd.Series({
        'IMPLIED_TEAM_POSS': x['IMPLIED_POSS'].sum()/5
    })
).reset_index()

opp_aggregated_stats = aggregated_stats.drop(columns=['OPP_TEAM'])
opp_aggregated_stats = opp_aggregated_stats.rename(columns={
    'TEAM': 'OPP_TEAM', 'IMPLIED_TEAM_POSS': 'IMPLIED_OPP_TEAM_POSS'
    })
aggregated_stats = aggregated_stats.merge(opp_aggregated_stats, on=['SEASON', 'DATE', 'OPP_TEAM'], how='left')

In [129]:
temp = temp.merge(aggregated_stats, on=['SEASON', 'DATE', 'TEAM', 'OPP_TEAM'], how='left')

In [130]:
temp['POSS'] = temp['IMPLIED_POSS'] * (temp['IMPLIED_TEAM_POSS'] + temp['IMPLIED_OPP_TEAM_POSS'])/(2*temp['IMPLIED_TEAM_POSS'])

In [131]:
aggregated_stats = temp.groupby(['SEASON', 'DATE', 'TEAM', 'OPP_TEAM']).apply(
    lambda x: pd.Series({
        'TEAM_OREB_CHANCES': (x['OREB(CH)/POSS_REAL'] * x['POSS']).sum()/5,
        'IMPLIED_TEAM_OREB': (x['OREB(CH)/POSS_REAL'] * x['POSS'] * x['OREB/CH_REAL']).sum(),
        'IMPLIED_TEAM_DREB': (x['OPP_OREB(CH)/POSS_REAL'] * x['POSS'] * x['DREB/CH_REAL']).sum()
    })
).reset_index()

opp_aggregated_stats = aggregated_stats.drop(columns=['OPP_TEAM'])
opp_aggregated_stats = opp_aggregated_stats.rename(columns={
    'TEAM': 'OPP_TEAM', 'TEAM_OREB_CHANCES': 'TEAM_DREB_CHANCES', 'IMPLIED_TEAM_OREB': 'IMPLIED_OPP_TEAM_OREB', 'IMPLIED_TEAM_DREB': 'IMPLIED_OPP_TEAM_DREB'
    })
aggregated_stats = aggregated_stats.merge(opp_aggregated_stats, on=['SEASON', 'DATE', 'OPP_TEAM'], how='left')

aggregated_stats['IMPLIED_TEAM_OREB_CHANCES'] = aggregated_stats['IMPLIED_TEAM_OREB'] + aggregated_stats['IMPLIED_OPP_TEAM_DREB']
aggregated_stats['IMPLIED_TEAM_DREB_CHANCES'] = aggregated_stats['IMPLIED_TEAM_DREB'] + aggregated_stats['IMPLIED_OPP_TEAM_OREB']

In [132]:
temp = temp.merge(aggregated_stats, on=['SEASON', 'DATE', 'TEAM', 'OPP_TEAM'], how='left')

In [133]:
temp['OREB/POSS'] = temp['OREB/CH_REAL'] * temp['OREB(CH)/POSS_REAL'] * (temp['TEAM_OREB_CHANCES']/temp['IMPLIED_TEAM_OREB_CHANCES'])
temp['DREB/POSS'] = temp['DREB/CH_REAL'] * temp['OPP_OREB(CH)/POSS_REAL'] * (temp['TEAM_DREB_CHANCES']/temp['IMPLIED_TEAM_DREB_CHANCES'])

In [134]:
aggregated_stats = temp.groupby(['SEASON', 'DATE', 'TEAM', 'OPP_TEAM']).apply(
    lambda x: pd.Series({
        'TEAM_POSS': x['POSS'].sum()/5,
        'TEAM_OREB': (x['OREB/POSS'] * x['POSS']).sum(),
        'TEAM_TOV': (x['TOV/POSS'] * x['POSS']).sum(),
        'IMPLIED_TEAM_ATT': (x['ATT/POSS'] * x['POSS']).sum()
    })
).reset_index()

In [135]:
temp = temp.merge(aggregated_stats, on=['SEASON', 'DATE', 'TEAM', 'OPP_TEAM'], how='left')

In [136]:
temp['TEAM_ATT'] = temp['TEAM_POSS'] + temp['TEAM_OREB'] - temp['TEAM_TOV']
temp['ATT/POSS_REAL'] = temp['ATT/POSS'] * (temp['TEAM_ATT']/temp['IMPLIED_TEAM_ATT'])

In [137]:
temp['PTS/ATT_REAL'] = temp['PTS/ATT'] + temp['PTS/ATT_DEF']
temp['PTS/POSS'] = temp['PTS/ATT_REAL'] * temp['ATT/POSS_REAL']

In [171]:
final = temp.groupby(['SEASON', 'DATE', 'TEAM', 'OPP_TEAM']).apply(
    lambda x: pd.Series({
        'PRED_TEAM_PTS': (x['PTS/POSS'] * x['POSS']).sum() + 0.62
    })
).reset_index()

In [172]:
opp_final = final.drop(columns=['OPP_TEAM'])
opp_final = opp_final.rename(columns={
    'TEAM': 'OPP_TEAM', 'PRED_TEAM_PTS': 'PRED_OPP_TEAM_PTS'
    })
final = final.merge(opp_final, on=['SEASON', 'DATE', 'OPP_TEAM'], how='left')

In [173]:
real = boxscores.groupby(['SEASON', 'DATE', 'TEAM', 'OPP_TEAM']).apply(
    lambda x: pd.Series({
        'REAL_TEAM_PTS': x['PTS'].sum()
    })
).reset_index()

opp_real = real.drop(columns=['OPP_TEAM'])
opp_real = opp_real.rename(columns={
    'TEAM': 'OPP_TEAM', 'REAL_TEAM_PTS': 'REAL_OPP_TEAM_PTS'
    })
real = real.merge(opp_real, on=['SEASON', 'DATE', 'OPP_TEAM'], how='left')

In [174]:
vegas = query_data.query_odds_data()
vegas = vegas.loc[vegas['PERIOD'] == 'Full Game']
vegas = vegas.drop(columns=['PERIOD'])

vegas['TOTAL'] = vegas['TOTAL'].replace(['-', 'PK'], np.nan)
vegas['POINTSPREAD'] = vegas['POINTSPREAD'].replace(['-', 'PK'], 0)

vegas['TOTAL'] = vegas['TOTAL'].astype(float)
vegas['POINTSPREAD'] = vegas['POINTSPREAD'].astype(float)

In [175]:
final = final.merge(real, on=['SEASON', 'DATE', 'TEAM', 'OPP_TEAM'])
final = final.merge(vegas, on=['DATE', 'TEAM'])

In [177]:
final['ONE'] = 1

feature_creation = FeatureCreation()
final = feature_creation.expanding_sum(
    df=final, group_col_names=['SEASON', 'TEAM'], col_name='ONE', new_col_name='GP'
)

In [245]:
final['PRED_TOTAL'] = final['PRED_TEAM_PTS'] + final['PRED_OPP_TEAM_PTS']
final['REAL_TOTAL'] = final['REAL_TEAM_PTS'] + final['REAL_OPP_TEAM_PTS']

final['MODEL_TOTAL_ERROR'] = final['PRED_TOTAL'] - final['REAL_TOTAL']
final['VEGAS_TOTAL_ERROR'] = final['TOTAL'] - final['REAL_TOTAL']

In [ ]:
final['PRED_PS'] = -1*(final['PRED_TEAM_PTS'] - final['PRED_OPP_TEAM_PTS'])
final['REAL_PS'] = -1*(final['REAL_TEAM_PTS'] - final['REAL_OPP_TEAM_PTS'])

final['MODEL_PS_ERROR'] = final['PRED_PS'] - final['REAL_PS']
final['VEGAS_PS_ERROR'] = final['POINTSPREAD'] - final['REAL_PS']

In [286]:
real_final = final.loc[final['GP'] > 10]
real_final = real_final.drop_duplicates(subset=['PRED_TOTAL'])

In [287]:
print(real_final['MODEL_TOTAL_ERROR'].abs().mean())
print(real_final['VEGAS_TOTAL_ERROR'].abs().mean())

14.682760481436143
14.1848


In [288]:
print(real_final['MODEL_PS_ERROR'].abs().mean())
print(real_final['VEGAS_PS_ERROR'].abs().mean())

10.322759119700294
9.56812729889653


In [299]:
blowout = real_final.loc[real_final['TOTAL'] > 240]
print(len(blowout.loc[blowout['REAL_TOTAL'] < blowout['TOTAL']]), len(blowout.loc[blowout['REAL_TOTAL'] > blowout['TOTAL']]))

14 6


In [283]:
print(len(final.loc[(final['POINTSPREAD'] < 0) & (final['REAL_TOTAL'] > final['TOTAL']) & (final['REAL_PS'] > final['POINTSPREAD'])]))
print(len(final.loc[(final['POINTSPREAD'] < 0) & (final['REAL_TOTAL'] > final['TOTAL']) & (final['REAL_PS'] < final['POINTSPREAD'])]))
print(len(final.loc[(final['POINTSPREAD'] < 0) & (final['REAL_TOTAL'] < final['TOTAL']) & (final['REAL_PS'] > final['POINTSPREAD'])]))
print(len(final.loc[(final['POINTSPREAD'] < 0) & (final['REAL_TOTAL'] < final['TOTAL']) & (final['REAL_PS'] < final['POINTSPREAD'])]))
print(len(final.loc[(final['POINTSPREAD'] < 0) & (final['REAL_TOTAL'] != final['TOTAL']) & (final['REAL_PS'] != final['POINTSPREAD'])]))

1720
1682
1719
1735
6859


In [284]:
test = real_final.loc[(real_final['PRED_TOTAL'] - real_final['TOTAL']).abs() > 5]

print(len(test.loc[test['PRED_TOTAL'] > test['TOTAL']]), len(test.loc[test['PRED_TOTAL'] < test['TOTAL']]))

test['MODEL_CORRECT_ON_TOTAL'] = test.apply(lambda row: np.sign((row['REAL_TOTAL'] - row['TOTAL']) * (row['PRED_TOTAL'] - row['TOTAL'])), axis=1)
print(len(test.loc[test['MODEL_CORRECT_ON_TOTAL'] == 1]), len(test.loc[test['MODEL_CORRECT_ON_TOTAL'] == -1]))
print(len(test.loc[test['MODEL_CORRECT_ON_TOTAL'] == 1])/(len(test.loc[test['MODEL_CORRECT_ON_TOTAL'] == 1]) + len(test.loc[test['MODEL_CORRECT_ON_TOTAL'] == -1])))

198 459
350 301
0.5376344086021505


In [285]:
test = real_final.loc[(real_final['PRED_PS'] - real_final['POINTSPREAD']).abs() > 5]

test['MODEL_CORRECT_ON_PS'] = test.apply(lambda row: np.sign((row['REAL_PS'] - row['POINTSPREAD']) * (row['PRED_PS'] - row['POINTSPREAD'])), axis=1)
print(len(test.loc[test['MODEL_CORRECT_ON_PS'] == 1]), len(test.loc[test['MODEL_CORRECT_ON_PS'] == -1]))
print(len(test.loc[test['MODEL_CORRECT_ON_PS'] == 1])/(len(test.loc[test['MODEL_CORRECT_ON_PS'] == 1]) + len(test.loc[test['MODEL_CORRECT_ON_PS'] == -1])))

336 316
0.5153374233128835


In [259]:
pd.set_option('display.max_columns', None)
display(real_final.loc[real_final['POINTSPREAD'].isnull()])

,SEASON,TEAM,DATE,OPP_TEAM,PRED_TEAM_PTS,PRED_OPP_TEAM_PTS,REAL_TEAM_PTS,REAL_OPP_TEAM_PTS,POINTSPREAD,MONEYLINE,TOTAL,ONE,GP,PRED_TOTAL,REAL_TOTAL,MODEL_TOTAL_ERROR,VEGAS_TOTAL_ERROR,PRED_PS,REAL_PS,MODEL_PS_ERROR,VEGAS_PS_ERROR
